In [75]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from skimage import data
from sklearn.svm import SVC
from collections import Counter

In [76]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [77]:
class KNN:
    def __init__(self, k=3):
        self.k = k

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def euclidean_distance(self, x1, x2):
        return np.sqrt(np.sum((x1 - x2)**2))

    def predict(self, X):
        y_pred = [self._predict(x) for x in X]
        return np.array(y_pred)

    def _predict(self, x):
        distances = [self.euclidean_distance(x, x_train) for x_train in self.X_train]
        k_indices = np.argsort(distances)[:self.k]
        k_nearest_labels = [self.y_train[i] for i in k_indices]
        most_common = Counter(k_nearest_labels).most_common(1)
        return most_common[0][0]

    def score(self, X, y):
        y_pred = self.predict(X)
        return accuracy_score(y, y_pred)


In [78]:
def load_images_from_folder(folder, num_descriptors=10, resize_shape=(128, 128)):
    images = []
    features_glcm = []
    features_fourier = []
    labels = []
    for label, class_folder in enumerate(os.listdir(folder)):
        class_path = os.path.join(folder, class_folder)
        for filename in os.listdir(class_path):
            img = cv2.imread(os.path.join(class_path, filename), cv2.IMREAD_GRAYSCALE)
            if img is not None:
                # Resize the image to a fixed shape
                img = cv2.resize(img, resize_shape)
                images.append(img)
                # Compute GLCM features
                #glcm_features = compute_glcm_features(img)
                #features_glcm.append(glcm_features)
                labels.append(label)
    # Check the length of all glcm features and crop them if necessary
    #min_length = min(len(feature) for feature in features_glcm)
    #features_glcm = [feature[:min_length] for feature in features_glcm]
    return np.array(images), np.array(labels)
x1, y1 = load_images_from_folder("/content/drive/MyDrive/archive2/Data/train")
x2, y2 = load_images_from_folder("/content/drive/MyDrive/archive2/Data/test")

In [79]:
def get_block(images, num_blocks_x, num_blocks_y):

    rows, cols = images.shape[1:]

    block_size_rows = rows // num_blocks_x
    block_size_cols = cols // num_blocks_y

    images_blocks=[]

    # Slice the image matrix into blocks

    for image in (images):
        sliced_blocks = []
        for i in range(num_blocks_x):
            for j in range(num_blocks_y):
             block = image[i * block_size_rows: (i + 1) * block_size_rows, j * block_size_cols: (j + 1) * block_size_cols]
             sliced_blocks.append(block)

        images_blocks.append(sliced_blocks)
    images_blocks=np.array(images_blocks)

    return images_blocks

def X_bar(img):

 XY=[]
 for i in range (len(img)):
    feature_vector=[]
    for block in range(len(img[i])):
        x_bar=0
        y_bar=0
        denomenator=np.sum(img[i][block])

        numerator1=0
        for x in range(len(img[i][block])):
            for y in range (len(img[i][block][x])):
                numerator1=x*img[i][block][x][y]+numerator1

        numerator2=0
        for y in range(img.shape[3]):
            for x in range (img.shape[2]):
                numerator2=y*img[i][block][x][y]+numerator2

        if denomenator != 0:
           x_bar = numerator1 / denomenator
           y_bar = numerator2 / denomenator
        else:
           x_bar = 0
           y_bar = 0
        feature_vector.append(x_bar)
        feature_vector.append(y_bar)
    feature_vector=np.array(feature_vector)

    XY.append(feature_vector)


 XY=np.array(XY)
 return XY
num_blocks_x =6
num_blocks_y =6
#X_cent_test=X_bar(X_test)

In [80]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

class AntColonyFeatureSelection:
    def __init__(self, X, y, pheromone_init=0.1, alpha=1, beta=2, evaporation_rate=0.5, iterations=100, ants=10):
        self.X = X
        self.y = y
        self.num_features = X.shape[1]
        self.pheromone = np.ones(self.num_features) * pheromone_init
        self.alpha = alpha
        self.beta = beta
        self.evaporation_rate = evaporation_rate
        self.iterations = iterations
        self.ants = ants

    def select_features(self, n_features):
        best_subset = None
        best_score = float('-inf')

        for _ in range(self.iterations):
            for ant in range(self.ants):
                subset = self.construct_solution()
                score = self.evaluate_subset(subset)
                if score > best_score:
                    best_score = score
                    best_subset = subset

                self.update_pheromone(subset, score)

            self.pheromone *= (1 - self.evaporation_rate)

        # Select the best 'n_features' based on their scores
        sorted_indices = sorted(range(len(best_subset)), key=lambda i: best_subset[i], reverse=True)
        best_subset = sorted_indices[:n_features]

        return best_subset


    def construct_solution(self):
        subset = []
        remaining_features = list(range(self.num_features))

        while remaining_features:
            probabilities = self.calculate_probabilities(remaining_features)
            probabilities /= np.sum(probabilities)  # Normalize probabilities
            feature = np.random.choice(remaining_features, p=probabilities)
            subset.append(feature)
            remaining_features.remove(feature)

        return subset

    def calculate_probabilities(self, remaining_features):
        probabilities = np.zeros(len(remaining_features))
        total_pheromone = np.sum(self.pheromone[remaining_features])

        if total_pheromone == 0:
            probabilities[:] = 1 / len(remaining_features)
        else:
            for i, feature in enumerate(remaining_features):
                probabilities[i] = (self.pheromone[feature] ** self.alpha) * ((1.0 / (total_pheromone)) ** self.beta) / total_pheromone

        return probabilities

    def evaluate_subset(self, subset):
        if len(subset) == 0:
            return float('-inf')

        knn = KNeighborsClassifier()
        X_subset = self.X[:, subset]
        knn.fit(X_subset, self.y)
        y_pred = knn.predict(X_subset)
        return accuracy_score(self.y, y_pred)

    def update_pheromone(self, subset, score):
        for feature in subset:
            self.pheromone[feature] += score  # Here, you can use the score or any other metric related to the feature subset


In [81]:
#centroid features
Xc = np.concatenate((x1, x2))
y = np.concatenate((y1, y2))

X_block=get_block(Xc,num_blocks_x, num_blocks_y)
X_cent=X_bar(X_block)
X_train, X_test, y_train, y_test = train_test_split(X_cent, y, test_size=0.15, random_state=42)

In [101]:
aco = AntColonyFeatureSelection(X_cent, y)
selected_features = aco.select_features(n_features=40)
print("Best subset:", selected_features)

Best subset: [3, 17, 19, 14, 5, 20, 37, 32, 52, 71, 2, 55, 36, 69, 39, 38, 43, 67, 65, 60, 6, 47, 46, 35, 22, 57, 51, 66, 0, 42, 58, 23, 50, 62, 28, 11, 8, 21, 4, 30]


In [102]:
# Reduce feature space based on selected features
X_selected_train = X_train[:, selected_features]
X_selected_test = X_test[:, selected_features]

In [103]:
# Reshape the input data
X_selected_train_flattened = X_selected_train.reshape(X_selected_train.shape[0], -1)
X_selected_test_flattened = X_selected_test.reshape(X_selected_test.shape[0], -1)

In [104]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Assuming X_selected_train_flattened contains your training data after feature selection


n_components = min(X_selected_train_flattened.shape[1], len(np.unique(y_train)) - 1)
lda = LinearDiscriminantAnalysis(n_components=n_components)

# Fit LDA to the training data
lda.fit(X_selected_train_flattened, y_train)

# Fit LDA to the training data and transform it
X_reduced_train = lda.fit_transform(X_selected_train_flattened, y_train)  # Assuming y_train contains the training labels

# Transform the test data using the fitted LDA
X_reduced_test = lda.transform(X_selected_test_flattened)
# Now, X_reduced_train and X_reduced_test contain the reduced feature representations
clf = KNN(k=3)
clf.fit(X_reduced_train, y_train)

# Predictions
predictions = clf.predict(X_reduced_test)
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.9625


In [107]:

import numpy as np

class CustomLDA:
    def __init__(self, n_components=None):
        self.n_components = n_components

    def fit(self, X, y):
        self.classes = np.unique(y)
        self.num_classes = len(self.classes)
        self.num_features = X.shape[1]

        # Compute class means
        self.class_means = np.array([np.mean(X[y == c], axis=0) for c in self.classes])

        # Compute within-class scatter matrix
        within_class_scatter = np.zeros((self.num_features, self.num_features))
        for c in self.classes:
            class_data = X[y == c]
            class_mean_diff = class_data - self.class_means[self.classes.searchsorted(c)]
            within_class_scatter += np.dot(class_mean_diff.T, class_mean_diff)

        # Compute between-class scatter matrix
        overall_mean = np.mean(X, axis=0)
        between_class_scatter = np.zeros((self.num_features, self.num_features))
        for c in self.classes:
            class_mean_diff = self.class_means[self.classes.searchsorted(c)] - overall_mean
            between_class_scatter += len(X[y == c]) * np.outer(class_mean_diff, class_mean_diff)

        # Solve the generalized eigenvalue problem
        eigvals, eigvecs = np.linalg.eig(np.linalg.inv(within_class_scatter).dot(between_class_scatter))
        eig_pairs = [(np.abs(eigvals[i]), eigvecs[:, i]) for i in range(len(eigvals))]
        eig_pairs = sorted(eig_pairs, key=lambda x: x[0], reverse=True)

        # Select top eigenvectors based on number of components
        if self.n_components is not None:
            eig_pairs = eig_pairs[:self.n_components]

        # Compute transformation matrix
        self.w_matrix = np.array([eig_pairs[i][1] for i in range(len(eig_pairs))])

    def transform(self, X):
        return np.dot(X, self.w_matrix.T)

lda = CustomLDA(n_components=2)
lda.fit(X_selected_train_flattened, y_train)
X_reduced_train = lda.transform(X_selected_train_flattened)
X_reduced_test = lda.transform(X_selected_test_flattened)
clf = KNN(k=3)
clf.fit(X_reduced_train, y_train)

# Predictions
predictions = clf.predict(X_reduced_test)
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.9875
